In [1]:
from tkinter import *
from PIL import ImageTk, Image
import cv2
import numpy as np
import json
from tensorflow.keras.models import load_model

In [2]:
f = open('Constants.json')
data = json.load(f)

labels = data['labels']

# number of classes
NUM_CLASSES = data['NUM_CLASSES']

# size of the images to be generated
IMG_SIZE = data['TRAIN_IMG_SIZE']

In [3]:
# loading the weights for the model
model=load_model('sign_lang.h5')

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 256)       2

In [5]:
def preprocess(frame):
    
    # convert to grayscale
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # defining the ROI
    x1 = int(0.5 * frame.shape[1])
    y1 = 5
    x2 = frame.shape[1] - 5
    y2 = int(0.7 * frame.shape[0])
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)

    # extracting the ROI
    roi = image[y1:y2, x1:x2]

    # blur the mask to help remove noise, then apply the
    # mask to the frame
    blur = cv2.GaussianBlur(roi, (5, 5), 5)

    # apply opening transformation to the mask
    # using an elliptical kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    image = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel)

    # use thresholding technique for segmenting the hand from the frame
    thr = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    ret, hand_img = cv2.threshold(thr, 150, 255, cv2.THRESH_BINARY_INV)
    
    return hand_img

In [6]:
class GUI:
    
    def __init__(self):
        self.current_word=''
        self.sentence=''
        self.charecter=''
        
        self.cap=cv2.VideoCapture(0)
        self.root=Tk()
        
        self.video_label=Label(self.root)
        self.video_label.grid(row=100,column=500,rowspan=200,columnspan=900)

        # Prediction Description Label
        self.predicted_desc=Label(self.root)
        self.predicted_desc.grid(row=600,column=10)
        self.predicted_desc.config(text="Character :",font=("Courier",40,"bold"))

        self.char = StringVar()

        # Prediction Label
        self.predicted=Label(self.root,textvariable=self.char)
        self.predicted.grid(row=600,column=60)
        self.predicted.config(textvariable=self.char,font=("Courier",40,"bold"))

        # Word Description Label
        self.word_desc=Label(self.root,text='Word:')
        self.word_desc.grid(row=1000,column=10)

        self.word = StringVar()

        # Prediction Label
        self.word_formed=Label(self.root,textvariable=self.word)
        self.word_formed.grid(row=1000,column=60)
        
        self.video_stream()

    def updateDepositLabel(self,*args):
        self.char.set(args[0])
        self.word.set(args[1])
        
    # function for video streaming
    def video_stream(self):
        
        ret,frame = self.cap.read()
        frame = cv2.flip(frame, 30)

        hand_img=preprocess(frame)
        self.prediction=self.predict(hand_img)
        if(len(self.current_word)!=0 and self.prediction=='blank'):
            self.current_word=''
        elif(self.prediction!='blank'):
            self.current_word+=self.prediction

        self.updateDepositLabel(self.prediction,self.current_word)
        cv2Img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2Img)
        imgtk = ImageTk.PhotoImage(image=img)
        self.video_label.imgtk = imgtk
        self.video_label.configure(image=imgtk)
        self.video_label.after(1, self.video_stream)
        
    def predict(self,hand_img):
        resized_hand_img=cv2.resize(hand_img, (IMG_SIZE, IMG_SIZE))
        x_test = np.array(resized_hand_img)
        x_test = x_test.reshape(1, IMG_SIZE, IMG_SIZE, 1)
        y_pred = np.argmax(model.predict(x_test))

        return labels[str(y_pred)]

In [7]:
gui=GUI()
gui.root.mainloop()

In [8]:
gui.cap.release()